# Spark structured streaming

### Turbine Analytics, 15.02.2017

W nawizaniu do poprzedniego głosowania


![polly](https://i.imgur.com/7XL7BBG.png)

Dziś pogadamy trochę o Spark structured streaming


## SPARK (STRUCTURED) STREAMING
### Trochę teorii o przetwarzaniu strumieni w Sparku

trochę o samym streaming'u

##### Batch vs Streaming

Przy przekształcaniu batch'owym dla nas dostępny "komplet" danych do obróbki.

Głowna idea w przekształceniu strumieniowym w tym żeby traktować dane jak tabełę do którę są dodawane ciągle są dodawane nowe linii.


Jakie są różnicy pomiędzy tymi trybami przekształeń danych:

**Stream**

1. Robi przekształcenie albo na jednym elemncie albo na kilku elementach (na oknie) który sytem dostał
2. Przekształcenia zazwyczaj proste (ze wzgłędu na potrzebe szybkiej odpowiedzi)
3. Obliczenia zazyczaj niezależne (w kontekscie okna lub rekordu)
4. Asynchroniczne zazwyczaj źródło danych nie interesuje się u nas czy otrzymaliśmy dane :)

**Batch**

1. Ma dostęp do wszystkich danych
2. Może sobie pozwolić na przekształcenia o dowolnym skomplikowaniu, Bo:
3. Job może trwać minuty (i godziny ;)


### Zalety przetwarzania strumieniowego

* Prostsze do zrozumienia — przetwarzania strumieniowe łatwiej pisać i łatwiej debugować
* near real time — przetwarzania strumieniowe pozwalają odpowiadać szybko -> sekundy lub milisekundy
* latwiejszy update danych — z natury obliczeń inkrementalnych.

### Wyzwania przetwarzania strumieniowego

* processowanie wiadomości które przyszli nie po kolei
* może być potrzeba trzeba przechowywać dużo informacji o stanie przetwarzania
* zapewnienie przetwarzania **1** raz nawet jeżeli coś padnie
* zapewnienie nizkiego czasu odpowiedzi
* połączenie danych z zewnętrznymi źródłami danych
* updejtowanie biznes logiki real time
* tranzakcyjny zapis do zewnętrz

### Dwa różnych API dla Streaming'u 

Spark ma bogatą historię wsparcia przetwarzania strumieniowego

W 2012 do Spark'u dołączył projekt SPark Streaming ze Swoim **DStreams** API.
To było jedno z pierwszych API (przynajmniej tak wyszło z mojego researchu :) którę dawało developerom możliwość 
użycia funkcji wysokopoziomowych, jakich jak _map_ i _reduce_.

Dotychczas jest Spark Streaming jest używany w wielu organizacji (patrząc na ilość pytań na StackOverflow).

Jednak _DStreams_ jak i RDD opierają się na obiektach Java/Python co w praktyce zawęża pole możliwych optymalizacji).


W 2016 więc powstał project Structured Streaming który używa DataFrame'y i Datasety. Structured Streaming API jest oznaczone jako stabilne w wersji Spark'a 2.2.0 więc jest gotowy do komercyjnego użycia.


To oznacza że on jest oparty na projekt Spark SQL (który pokazywałem na poprzednim spotkaniu). A to oznacza że **można używać ten sam kod dla batchowych przekształcen jak i dla strumieniowych**

Structured streaming pozwala na:
* **agregacje** na strumieniach
* **uzycia okien czasowych**
* joinować strumienie batchowe i strumieniowe.

Wszystkie optymalizację, o których mówiłem na poprzednim spotkaniu, czyli query analyzer, cost optimizer

![query analyzer](https://i.imgur.com/j6NYnac.png)

![cost optimizer](https://i.imgur.com/nja8qJo.png)

oraz kastomowy serializator, off heap data storage i td. cą dostępne w structured streaming.


### Przetwarzanie ciągłę vs Micro-batch przetwarzanie


**przetwarzenie ciągłe**
![one record at a time](https://i.imgur.com/pkQJouq.png)

* pozwala na najmnieszy czas odpowiedzi. Natomiast takie systemy mają miejszy throughput, dlatego że mają większy overhead (wysyłanie każdej wiadomości to wywołąnie funkcji OS i która robi pakiet sieciowy i td)
* fixed topology. Czyli trudno zmienić tor przekształcenie w runtime.






**przetwarzenie micro-batch**
Akumuluje wiadomości w małych batchach (na przykład po 300 ms) i procesuje ten batch job w Spark'u

![micro-batch](https://i.imgur.com/b5jGg7B.png)

* większa wydajność systemu
* pozwazala używać tych optymalizacji o których mówiłem wcześniej.
* potrzebują mniej węzłów żeby przeształcić ten sam wolumen danych.


Co wybrać:
* Jeżeli 100 ms to nie akceptowalne opóźnienie to lepiej wybrać przetwarzanie ciągłe.

**Stare API**

In [ ]:
val conf = new SparkConf().setAppName("SparkStreamingExampleApp").setMaster("local[*]")
val ssc = new StreamingContext(conf, Seconds(5))

///ssc.textFileStream("path")
///ssc.queueStream(queue: mutable.Queue[RDD[T]], oneAtATime: Boolean = true): InputDStream[T]

**Nowe API**

In [5]:
import org.apache.spark.sql.DataFrame

//batch df
val df: DataFrame = spark.read.csv("sales")//options
df.isStreaming //return false

val streamingDf:DataFrame = spark.readStream.format("rate").load
streamingDf.isStreaming//returns true



true